# EDS Block Experimentation - Context Aware

This notebook works in both JSLab (Node.js) and browser environments.

## Environment Detection

- **JSLab/Node.js**: Sets up jsdom virtual DOM and helper functions
- **Browser (ipynb-viewer)**: Uses native browser APIs directly

## Important Notes

**Always**: Run First Javascript Cell first to initialize the environment
**Pure EDS blocks** - Works best with vanilla JavaScript blocks
**Context-aware execution** - Automatically detects Node.js vs browser
**Visual output** - Generates live preview HTML with iframe controls, in Browser
**Iframe previews** - Open `-live-preview.html` files for interactive testing, in Node

## Initialization Output

When you run the first JavaScript cell, you'll see:

```
✅ Environment (Node.js) setup
```

Or in browser:

```
✅ Environment (Browser) setup
```

In [ ]:
// ============================================================================
// SETUP: One-line initialization! (works in both JSLab and Browser)
// ============================================================================

return (async () => {
  const isNode = typeof process !== 'undefined' && process.versions?.node;
  const helpersPath = isNode ? './scripts/ipynb-helpers.js' : '/scripts/ipynb-helpers.js';
  const { initialize } = await import(helpersPath);
  await initialize();
  
  // Return simple success message
  const context = isNode ? 'Node.js' : 'Browser';
  return `✅ Environment (${context}) setup`;
})();

## Part 1: Simple Tests

Test basic DOM functionality that works in both environments.

In [ ]:
// Context-aware DOM test using unified API (no ternary operators!)
(async () => {
const testDiv = doc.createElement('div');
testDiv.textContent = 'Hello from ' + (isNode ? 'Node.js (JSLab)' : 'Browser (ipynb-viewer)') + '!';
testDiv.className = 'test';
testDiv.style.cssText = 'padding: 20px; background: #e3f2fd; border-radius: 8px; color: #1976d2; font-weight: bold;';

console.log('✓ Created element:', testDiv.outerHTML);

// Return for display
return testDiv.outerHTML;
})();

## Part 2: Testing Blocks with Content

Test blocks that require specific content structures.

In [ ]:
// Context-aware block testing using unified API (no ternary operators!)
(async () => {
// Example 1: Test HelloWorld block (no content needed)
const helloBlock = await testBlockFn('helloworld');
console.log('✓ HelloWorld block created');

return helloBlock.outerHTML;
})();

In [ ]:
// Example 2: Test Accordion block - BEFORE and AFTER
(async () => {
const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks transform DOM elements using JavaScript decoration.</div>
  </div>
`;

// BEFORE
const before = doc.createElement('div');
before.className = 'accordion';
before.innerHTML = accordionContent;
const beforeHTML = before.innerHTML.trim().substring(0, 100);

// Suppress console during decoration
const originalLog = console.log;
console.log = () => {};

// AFTER - using unified API
const after = await testBlockFn('accordion', accordionContent);

// Restore console
console.log = originalLog;

const afterHTML = after.innerHTML.trim().substring(0, 100);

return `BEFORE:\n${beforeHTML}...\n\nAFTER:\n${afterHTML}...\n\nTransformed: ${before.children.length} divs → ${after.querySelectorAll('details').length} details`;
})();

## Part 3: Visual Output with Iframe

Create visual previews with iframe controls - works in both Node.js and browser!

In [ ]:
// Context-aware iframe preview using unified API (MUCH simpler!)
(async () => {
const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
  <div>
    <div>Why use ipynb-viewer?</div>
    <div>The ipynb-viewer block lets end users interact with executable notebooks directly on your EDS site.</div>
  </div>
`;

// Single function works in both environments!
return await showPreview('accordion', accordionContent);
})();

## Part 4: Discover Available Blocks

List all blocks available for testing in this project.

In [ ]:
// Context-aware block discovery using global flags
(async () => {
if (isNode) {
  // Node.js: Use fs to list directories
  const fs = require('fs').promises;
  const blocksDir = './blocks';
  
  const entries = await fs.readdir(blocksDir, { withFileTypes: true });
  const blocks = entries
    .filter(entry => entry.isDirectory())
    .map(entry => entry.name)
    .sort();
  
  console.log(`Found ${blocks.length} blocks:\n`);
  console.log(blocks.join('\n'));
  
  return blocks;
} else {
  // Browser: Use fetch to get block list (if available)
  console.log('⚠ Block discovery requires Node.js/JSLab environment');
  console.log('ℹ️  In browser, blocks must be specified manually');
  
  // Return common blocks
  const commonBlocks = ['accordion', 'cards', 'columns', 'fragment', 'header', 'footer', 'hero'];
  console.log('Common blocks:', commonBlocks.join(', '));
  
  return commonBlocks;
}
})();

## Part 5: Interactive Calculations

Pure JavaScript that works in both environments.

In [ ]:
// Simple calculation - works everywhere
const a = 10;
const b = 20;
const sum = a + b;

console.log('Sum:', sum);
console.log('Product:', a * b);
console.log('Average:', sum / 2);

sum

In [ ]:
// Array operations - works everywhere
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
const sum = numbers.reduce((acc, n) => acc + n, 0);

console.log('Original:', numbers);
console.log('Doubled:', doubled);
console.log('Sum:', sum);

doubled

## Part 6: String Manipulation

In [ ]:
// Text transformations - works everywhere
const text = 'hello world from jupyter';
const upper = text.toUpperCase();
const titleCase = text.replace(/\b\w/g, c => c.toUpperCase());

console.log('Original:', text);
console.log('Uppercase:', upper);
console.log('Title Case:', titleCase);

titleCase

## Quick Reference - Part 1

### Global Environment Flags (Set by First Javascript Cell)

After running the fist javascript cell, these global flags are available in all subsequent cells:

**Node.js (JSLab):**
```javascript
global.isNode      // true
global.isBrowser   // false
```

**Browser (ipynb-viewer):**
```javascript
window.isNode      // false
window.isBrowser   // true
```

### Unified API

First Javascript Cell also sets up a **unified API** that works identically in both environments:

**Core References:**
```javascript
doc              // Document object (no ternary needed!)
testBlockFn      // Test block function (no ternary needed!)
createPreviewFn  // Create iframe preview HTML
showPreview      // Create/open preview (Node: saves files, Browser: opens popup)
```

**Simple Usage (works in both Node.js and browser):**
```javascript
// Create DOM elements
const div = doc.createElement('div');

// Test blocks
const block = await testBlockFn('blockname', '<div>content</div>');

// Create and show preview (automatically adapts to environment!)
await showPreview('blockname', '<div>content</div>');
```


```javascript
// Just use the globals directly!
const block = await testBlockFn('blockname', '<div>content</div>');
```

### Browser Compatibility

**Wrap async code in IIFE:**
```javascript
(async () => {
  const block = await testBlockFn('blockname', '<div>content</div>');
  return block.outerHTML;
})();
```

## Quick Reference - Part 2

### Node.js Helpers (JSLab only)

**Test blocks (returns DOM element):**
```javascript
const block = await global.testBlock('blockname', '<div>content</div>');
block.outerHTML
```

**Save with Live Preview (creates iframe wrapper):**
```javascript
// Creates TWO files:
// 1. blockname-preview.html (actual styled block)
// 2. blockname-live-preview.html (iframe wrapper with controls)
await global.saveBlockHTML('blockname', '<div>content</div>');

// Custom filename
await global.saveBlockHTML('blockname', '<div>content</div>', 'my-test.html');

// Disable live preview
await global.saveBlockHTML('blockname', '<div>content</div>', null, { livePreview: false });
```

**Create iframe preview HTML:**
```javascript
const previewHTML = global.createIframePreview('blockname', '<div>block html</div>');
```

**Load styles manually:**
```javascript
await global.loadBlockStyles('blockname');
```

### Browser Helpers (ipynb-viewer)

**Test blocks:**
```javascript
const block = await window.testBlock('blockname', '<div>content</div>');
const container = window.displayBlock(block);
```

**Create and open iframe preview:**
```javascript
// Generate iframe HTML
const previewHTML = window.createIframePreview('blockname', '<div>block html</div>');

// Open preview in new window
window.openIframePreview('blockname', '<div>block html</div>');
```

### Live Preview Features
- 🔴 Dark themed wrapper with controls
- ↻ Refresh button to reload preview
- ✕ Close button (or press ESC key)
- Status bar showing file location
- Fullscreen display with scrolling

## Quick Reference - Part 3

### Tips
 **First Javascript Cell** sets up unified API - use `doc`, `testBlockFn`, `showPreview` directly
 **No ternary operators needed** - unified API works in both environments
**Simpler code** - `showPreview('blockname', content)` replaces 20+ lines of conditional logic
**JSLab mode**: Full block testing with jsdom, saves HTML files to disk
**Browser mode**: Direct DOM interaction, opens preview in new window
**Live preview**: Works in both environments with different mechanisms
**Context-aware**: Write code once, runs in both environments
**Node.js**: Files saved to `ipynb-tests/` directory
**Browser**: Preview opens in popup window via Blob URL
**Browser async**: Wrap `await` in `(async () => { ... })()` IIFE

### Environment Capabilities

| Feature | JSLab (Node.js) | Browser (ipynb-viewer) |
|---------|----------------|------------------------|
| DOM Creation | ✅ jsdom | ✅ Native |
| Block Testing | ✅ Full | ⚠️ Limited |
| File I/O | ✅ Yes | ❌ No |
| Live Preview | ✅ Saved files | ✅ Popup window |
| Iframe Preview | ✅ Yes | ✅ Yes |
| Calculations | ✅ Yes | ✅ Yes |
| String Ops | ✅ Yes | ✅ Yes |
| Top-level await | ✅ Yes | ❌ No (use IIFE) |
| Global Flags | ✅ global.isNode, global.isBrowser | ✅ window.isNode, window.isBrowser |
| Unified API | ✅ doc, testBlockFn, showPreview | ✅ doc, testBlockFn, showPreview |